In [6]:
!pip install reportlab ipywidgets pandas

import os
import hashlib
import json
import csvq
from datetime import datetime
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from IPython.display import display, FileLink
from ipywidgets import FileUpload, Button, HBox, Output

IOC_HASHES = {
    "e99a18c428cb38d5f260853678922e03": "Malware Sample 1",
    "d41d8cd98f00b204e9800998ecf8427e": "Empty File (Suspicious)"
}

class CyberTriageTool:
    def __init__(self, evidence_path):
        self.evidence_path = evidence_path
        self.results = []
        self.score_threshold = 0.7
        self._prepare_environment()

    def _prepare_environment(self):
        """Create evidence directory if it doesn't exist"""
        if not os.path.exists(self.evidence_path):
            os.makedirs(self.evidence_path)
            print(f"Created evidence directory: {self.evidence_path}")

    def collect_data(self):
        """Import and process evidence from directory"""
        print(f"\n Collecting data from {self.evidence_path}...")
        for root, _, files in os.walk(self.evidence_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    file_hash = self.calculate_hash(file_path)
                    self.results.append({
                        "file": file_path,
                        "hash": file_hash,
                        "timestamp": datetime.fromtimestamp(os.path.getmtime(file_path)).isoformat(),
                        "size": os.path.getsize(file_path),
                        "suspicious": self.check_ioc(file_hash),
                        "score": self.simulate_ai_score(file_hash)
                    })
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
        print(f" Collected {len(self.results)} files")

    def calculate_hash(self, file_path):
        """Calculate MD5 hash of a file"""
        hasher = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hasher.update(chunk)
        return hasher.hexdigest()

    def check_ioc(self, file_hash):
        """Check hash against known IOCs"""
        return IOC_HASHES.get(file_hash, None)

    def simulate_ai_score(self, file_hash):
        """Generate simulated risk score"""
        return 0.9 if file_hash in IOC_HASHES else 0.3

    def analyze(self):
        """Run full analysis"""
        self.collect_data()
        return pd.DataFrame(self.results)

    def generate_report(self, format="pdf"):
        """Generate analysis report"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"report_{timestamp}.{format}"
        
        if format == "pdf":
            self._generate_pdf_report(filename)
        elif format == "json":
            self._generate_json_report(filename)
        elif format == "csv":
            self._generate_csv_report(filename)
        
        print(f"\n📄 Report generated: {filename}")
        display(FileLink(filename))
        return filename

    def _generate_pdf_report(self, filename):
        doc = SimpleDocTemplate(filename, pagesize=letter)
        styles = getSampleStyleSheet()
        story = [Paragraph("Cyber Triage Tool Report", styles["Title"])]
        story.append(Spacer(1, 12))
        
        for result in self.results:
            text = [
                f"<b>File:</b> {result['file']}",
                f"<b>Hash:</b> {result['hash']}",
                f"<b>Score:</b> {result['score']:.2f}"
            ]
            if result["suspicious"]:
                text.append(f"<font color='red'><b>Suspicious:</b> {result['suspicious']}</font>")
            
            story.append(Paragraph("<br/>".join(text), styles["BodyText"]))
            story.append(Spacer(1, 12))
        
        doc.build(story)

    def _generate_json_report(self, filename):
        with open(filename, "w") as f:
            json.dump(self.results, f, indent=4)

    def _generate_csv_report(self, filename):
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)

# Create GUI components for Jupyter
uploader = FileUpload(multiple=True, description="Choose files")
process_btn = Button(description="Analyze Files", button_style='success')
output = Output()
test_dir = "./test_evidence"

def on_process_click(_):
    with output:
        output.clear_output()
        
        # Save uploaded files
        if uploader.value:
            tool = CyberTriageTool(test_dir)
            # Process each uploaded file
            for file_info in uploader.value:
                file_name = file_info['name']
                file_content = file_info['content']
                with open(os.path.join(test_dir, file_name), "wb") as f:
                    f.write(file_content)
            
            print(" Uploaded files:")
            display(pd.DataFrame([f['name'] for f in uploader.value], columns=["Uploaded Files"]))
            
            # Run analysis
            print("\n Analyzing files...")
            df = tool.analyze()
            print("\n🔬 Analysis Results:")
            display(df)
            
            # Generate reports
            print("\n Generating reports...")
            tool.generate_report("pdf")
            tool.generate_report("csv")
            tool.generate_report("json")
            print("\n Analysis complete!")

# Connect button click handler
process_btn.on_click(on_process_click)

# Display UI components
display(HBox([uploader, process_btn]), output)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Output()